In [1]:
# I need to search with the given link ('?ref=category'), look for 
# video with match = re.search("data-has-video=\"true\"",project_raw)

In [2]:
import requests
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode

In [3]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [7]:
with con:  
    cur = con.cursor()
    cur.execute("SHOW DATABASES")
    
    rows = cur.fetchall()
    for row in rows:
        print row

('information_schema',)
('kickscrape',)
('kickstarter',)
('mysql',)
('performance_schema',)
('test',)
('testdb',)
('world_innodb',)


In [32]:
#Create and describe my new table
with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS Has_video")
    cur.execute("CREATE TABLE Has_video(Id INT PRIMARY KEY AUTO_INCREMENT,\
                 Video_present TINYINT,\
                 Main_page_text LONGTEXT)")
    
    cur.execute("DESCRIBE Has_video")
    rows = cur.fetchall()
    
    for row in rows:
        print row[0:3]
    cur.close()

('Id', 'int(11)', 'NO')
('Video_present', 'tinyint(4)', 'YES')
('Main_page_text', 'longtext', 'YES')


In [10]:
# Now make a dataframe of all primary keys and URL, loop through them

In [12]:
with con:
    cur = con.cursor()
    cur.execute("Select Id,URL FROM Project_info");
    id_url_rows = cur.fetchall()
    cur.close()

In [17]:
id_url_df = pd.DataFrame(np.asarray(id_url_rows),columns=['Id','URL'])

In [19]:
id_url_df.tail()

,Id,URL
191379,191380,https://www.kickstarter.com/projects/207940292...
191380,191381,https://www.kickstarter.com/projects/184521459...
191381,191382,https://www.kickstarter.com/projects/209708895...
191382,191383,https://www.kickstarter.com/projects/thelifecu...
191383,191384,https://www.kickstarter.com/projects/reaper889...


In [22]:
n_projects = np.shape(id_url_df)[0]
print n_projects

191384


In [ ]:
with con:
    cur = con.cursor()
    for i in range(5,n_projects):
        
        if not i%1000:
            print i
        
        primary_key = id_url_df.ix[i,'Id']
        url = id_url_df.ix[i,'URL']

        has_video = 0
        project_request = requests.get(url)
        project_raw = unidecode(project_request.text)
        match = re.search("video_height",project_raw)
        if match:
            has_video = 1
            
        cur.execute("INSERT INTO Has_video () VALUES(%s,%s,%s)",\
            [primary_key,has_video,project_raw])
        
        con.commit()
    
    cur.close()